# Using PhysiCOOL with third-party libraries

The `BlackBoxModel` class can be used in combination with other libraries to perform parameter studies and optimization routines that have still not been implemented in PhysiCOOL.

Here, we describe how to couple our approach with the [psweep](https://github.com/elcorto/psweep) library, which can be used to set up and run parameter studies. The PhysiCell project used is the one described in the motility example. 

A Pandas DataFrame is created by `psweep` with the model outputs for each parameter set and some additional information (run_id, run_time...). Results are also stored in a folder called `calc`.

This approach can be used with other libraries, as users see fit.

> **Note for users running this notebook locally:** You will need to install psweep (using pip install)

## Compiling the PhysiCell project

In [2]:
from physicool import optimization as opt

# Compiles the project and creates a black box object for it
opt.compile_project() 

## Running the `psweep` parameter study

In [3]:
from physicool. optimization import PhysiCellBlackBox
from physicool.updaters import CellUpdater, update_motility_values
from physicool.processing import get_final_y_position

import psweep as ps
import shutil

# Compiles the project and creates a black box object for it
# opt.compile_project()    
black_box = PhysiCellBlackBox()

# Define the updater we want to use (change motility data)
updater = CellUpdater(updater_function=update_motility_values,
                     config_path="config/PhysiCell_settings.xml", 
                     cell_definition_name="default")

# Assign the updater and processor to the black box
black_box.updater = updater
black_box.processor = get_final_y_position

def func(pset): 
    """
    Runs the black box model with the values selected by psweep.
    """
    shutil.rmtree("temp", ignore_errors=True)
    metric = black_box.run(pset)
    return {'result': metric}

# Choose parameters
# (Creates a grid [["speed": 1.0, "bias": 0.2],
#                  ["speed": 1.0, "bias": 0.6]])
a = ps.plist('migration_bias', [0.2, 0.6])
b = ps.plist('speed', [1.0])
params = ps.pgrid(a,b)

# Get a DataFrame with all the simulation results
# psweep will run func() with each pair of the params grid
df = ps.run_local(func, params)
df.head()

,migration_bias,speed,_pset_hash,_run_id,_pset_id,_calc_dir,_time_utc,_pset_seq,_run_seq,result,_pset_runtime
0,0.2,1.0,cd3eeb3b2452a69eac62b2d2d76d9f29c1d19be4,bb11e8ba-d24c-431f-97a9-c6ef22262eff,99548b15-f01b-4a14-a35f-ddca078d750e,calc,2023-02-08 22:08:57.772333145,0,0,"[-491.465719040412, -152.6958488711255, -490.8...",2.476987
1,0.6,1.0,5a9a45aab62d3a8fed2acdc73b803e96429a24f4,bb11e8ba-d24c-431f-97a9-c6ef22262eff,a087fed7-2241-417c-8d2b-5bece85525c8,calc,2023-02-08 22:09:00.252892494,1,0,"[-236.23958073395102, -107.25965253084702, -25...",2.344300
